#**PyUnloader**

Este es un programa que exporta los datos contenidos en una base de datos SYBASE SQL Anywhere version 7 y produce arvhivos con sentencias DDL en donde se crer tablas, vistas, se definen llaves primarias, indices y relaciones entre tablas.

Con este objetivo se realizan queries sobre las tablas del sistema :

**SYSUSERS** : contiene la informacion de los usuarios creados en la base de datos

**SYSTABLE** : contiene informacion de las tablas y vistas 

**SYSCOLUMNS** : contiene infromacion sobre las columnas de cada tabla

**SYSINDEXES** :  contiene infromacion sobre los indices y constraints creados para las tablas


In [50]:
import pyodbc  

DataSource = "DSN=BaseSQL32"
ExportFile = "PyUnload"
ExportDataDir = "C:\\PyUnload2\\PyUnload\\Data"
Usuario = 'PRG'
Filtro = ["Productos"]

In [54]:
sql_user = ''' 
DROP USER "PRG"

GRANT CONNECT TO "PRG"   IDENTIFIED BY 
ENCRYPTED '\\x63\\x4A\\xC1\\xFB\\xC0\\xDD\\x71\\xE2\\xD4\\xD4\\x79\\x1F\\x43\\x3E\\xA1\\x82\\xF5\\x2C\\xFF\\xB7\\xB4\\xBF\\x57\\x0F\\x19\\x40\\xF6\\x09\\x13\\x6A\\x31\\x2F\\xE4\\x5A\\x93\\x22';

COMMENT ON USER "PRG" IS 'Usuario usado por los programas';

GRANT GROUP TO "PRG";

GRANT RESOURCE, DBA, REMOTE DBA TO "PRG";

'''

file = open(ExportFile +'-'+ Usuario +'.sql', "w")
file.write(sql_user)
file.close()

In [39]:
# Estrae la lista de TABLAS y VISTAS de un propietario especifico
 
def getSysTables(creator, tipoObjeto, filtro):

    lista_tablas = []
    creator_id = ""
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()
    
    sql_usuarios  = "SELECT * FROM SYSUSERS WHERE NAME = " + "'" + creator + "'"
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        creator_id=row.suid

    if len(filtro)==0:
        sql_tablas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = '" + tipoObjeto 
        sql_tablas = sql_tablas + "' AND CREATOR=" + str(creator_id)
        sql_tablas = sql_tablas + " ORDER BY TABLE_NAME "
    else:
        sql_tablas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = '" + tipoObjeto 
        sql_tablas = sql_tablas + "' AND CREATOR=" + str(creator_id)
        for fil in filtro:
            sql_tablas = sql_tablas + " AND TABLE_NAME='" + fil +"'"
        sql_tablas = sql_tablas + " ORDER BY TABLE_NAME "

    
    cursor.execute(sql_tablas)
    rows = cursor.fetchall()
    
    print (sql_tablas)
    print(len(rows))

    for row in rows:
        tbl={}
        tbl["table_creator"] = creator
        tbl["table_id"] = row.table_id
        tbl["table_count"]= row.count
        tbl["table_name"]= row.table_name
        lista_tablas.append(tbl)

    cursor.close()
    conexionDB.close()

    return lista_tablas

In [40]:
# Se crea una lista que contiene la informacion principal de cada TABLA : Creador, Id y Nombre
#
#{'table_creator': 'PRG', 'table_id': 358, 'table_count': 0, 'table_name': 'ABC'}
#{'table_creator': 'PRG', 'table_id': 379, 'table_count': 4, 'table_name': 'ABCObsoletos'}
#{'table_creator': 'PRG', 'table_id': 380, 'table_count': 3, 'table_name': 'ABCUnidades'}
#...
#for usr in Usuarios:
lista_tablas = getSysTables(Usuario, 'BASE',Filtro)
file = open("Tablas-" + Usuario + ".txt", "a")
file.write('\n'.join(str(lt) for lt in lista_tablas))
file.close()

# Se crea un diccionario con la informacion principal de cada VISTA : Creador, Id y Nombre
#
#{'table_creator': 'PRG', 'table_id': 451, 'table_count': 0, 'table_name': 'ALFA1'}
#{'table_creator': 'PRG', 'table_id': 8899, 'table_count': 0, 'table_name': 'CajaDM1'}
#{'table_creator': 'PRG', 'table_id': 442, 'table_count': 0, 'table_name': 'ClienteFicha'}
#for usr in Usuarios:
lista_vistas = getSysTables(Usuario, 'VIEW', Filtro)
file = open("Vistas-" + Usuario + ".txt", "w")
file.write('\n'.join(str(lt) for lt in lista_tablas))
file.close()

SELECT * FROM SYSTABLE WHERE TABLE_TYPE = 'BASE' AND CREATOR=101 AND TABLE_NAME='Productos' ORDER BY TABLE_NAME 
1
SELECT * FROM SYSTABLE WHERE TABLE_TYPE = 'VIEW' AND CREATOR=101 AND TABLE_NAME='Productos' ORDER BY TABLE_NAME 
0


In [41]:
# Constuye una lista con la informacion de las columnas de la tabla indicada en el parametro

def getSysColumns(table_name):
    columnas = []
 
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    sql_usuarios  = "SELECT * FROM SYSCOLUMNS WHERE TNAME = " + "'" + table_name + "'" + " ORDER BY COLNO"
    
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        obj = {}
        obj["col_name"]=row.cname
        obj["col_type"]=row.coltype
        obj["col_null"]=row.nulls
        obj["col_length"]=row.length
        obj["col_inPK"]=row.in_primary_key
        columnas.append(obj)

    cursor.close()
    conexionDB.close()

    return columnas

In [42]:
# Genera la sentencia DDL de CREATE TABLE de la tabla indicada en los parametros

def createTable(creator,table_name,columns, drop_table):
    sentenciaDDL = ""

    if drop_table == True:
        sentenciaDDL = "DROP TABLE " + table_name +"\"\n" 
    
    sentenciaDDL = "CREATE TABLE " + "\"" + creator + "\".\"" + table_name +"\"\n" 
    sentenciaDDL = sentenciaDDL + "(" +"\n" 
    for col in columns:
        sentenciaDDL = sentenciaDDL + "\t" + col["col_name"] 
        if col["col_length"] != "": 
            if col["col_type"] != "long varchar" and col["col_type"] != "integer":
                sentenciaDDL = sentenciaDDL + " " + col["col_type"] + "(" + str(col["col_length"]) + ")" 
            else:
                sentenciaDDL = sentenciaDDL + " " + col["col_type"]
        if col["col_null"] == "N":
            sentenciaDDL = sentenciaDDL + " NOT NULL" +",\n"
        else:
            sentenciaDDL = sentenciaDDL + ",\n"

    sentenciaDDL =  sentenciaDDL+ ")" + ";\n\n"

    return sentenciaDDL

In [43]:
# Escribe sentencias DDL de CREATE TABLE para las tablas encontradas

todas_columnas =[]

#for usr in Usuarios:
file = open(ExportFile +'-'+ Usuario +'.sql', "a")
for col in lista_tablas:
    columnas = getSysColumns(col["table_name"])
    sentenciaDDL = createTable(col["table_creator"], col["table_name"], columnas, True)
    file.write(sentenciaDDL)

    todas_columnas.append("\n" + col["table_name"])
    todas_columnas.append('\n'.join(str(x) for x in columnas))
file.close()

file = open("Columnas-" + Usuario +'.txt', "w")
file.write(('\n'.join(str(x) for x in todas_columnas)))
file.close()


In [44]:
# Extrae la lista de indices de la tabla indicada en los parametros

def getSysIndexes(table_name):

    indices = []
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    sql_indices  = "SELECT * FROM SYSINDEXES WHERE INDEXTYPE = 'Non-unique' AND TNAME =" + "'" + table_name + "' order by iname"   

    cursor.execute(sql_indices)
    rows = cursor.fetchall()
    
    for row in rows:
        obj={}
        obj["creator"]=row.creator
        obj["table_name"]=row.tname
        obj["ind_name"]=row.iname
        colnames=row.colnames
        obj["columns"]=colnames.split(',')
        indices.append(obj)

    cursor.close()
    conexionDB.close()

    return indices

In [45]:
# Genera sentencia DDL de CREATE INDEX para la tabla indicada 

def createIndexes(indices):

    sentenciaCompleatDDL = "" 
    for ind in indices:
        sentenciaDDL = ""
        sentenciaDDL = "CREATE INDEX " + "\"" + ind["ind_name"] + "\" ON " + "\"" + ind["creator"] + "\".\"" + ind["table_name"] +"\"\n" 
        sentenciaDDL = sentenciaDDL + "(" +"\n" 
        for col in ind["columns"]:
            sentenciaDDL = sentenciaDDL + "\t"  + col + ",\n"
        sentenciaDDL = sentenciaDDL + ");" +"\n"
        sentenciaCompleatDDL = sentenciaCompleatDDL + sentenciaDDL
        
    return sentenciaCompleatDDL
    

In [46]:
# Escribe sentencia DDL de CREATE INDEX para todas las tablas encontradas

file = open(ExportFile +'-'+ Usuario +'.sql', "a")
for tbl in lista_tablas:
    indices_tabla =  getSysIndexes(tbl["table_name"])
    indicesDDL = createIndexes(indices_tabla)
    file.write(indicesDDL)

file.close()

In [49]:
def exportData(table_name, table_id, max_rows):

    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()
    
    if max_rows!=0:
        sql  = "select top " + str(max_rows) + " * from " + table_name
    else:
        sql  = "select * from " + table_name

    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  ExportDataDir + str(table_id) + ".dat"
    
    if len(rows)>0:    
        file = open(archivo_dat, "w")
        for row in rows:
            r = str(row)
            r = r.replace('(','')
            r = r.replace(')','')
            r = r.replace('Decimal','')
            file.write(r+"\n")
        file.close()
    else:
        return archivo_dat, False
    
    return archivo_dat , True

In [48]:
# Exporta los datos de contenidos en las tablas 

archivos_dat = []
for col in lista_tablas:
    exportedFile, exportResult = exportData(col["table_name"], col["table_id"],100)
    if exportResult:
        print (exportedFile)
        archivos_dat.append(col)

TypeError: can only concatenate str (not "int") to str

In [ ]:
def insertData(dat, archivo_sql):

    print(dat)
   
    archivo_dat = str(dat["table_id"]) + '.dat'

    #print(archivo_dat)
    file = open(archivo_sql, "a")

    sentenciaInsert = "INPUT INTO " + "\"" + dat["table_creator"] + "\".\"" + dat["table_name"] +"\"\n"
    sentenciaInsert = sentenciaInsert + "FROM " +  "\"" + archivo_dat + "\"\n"  
    sentenciaInsert = sentenciaInsert +  "FORMAT ASCII\n"
    sentenciaInsert = sentenciaInsert +  "BY ORDER;\n\n"
    
    file.write(sentenciaInsert)
    file.close()

    return True 

In [ ]:
for dat in archivos_dat:
    insertData(dat, ExportFile)